In [1]:
import numpy as np
import scipy.linalg as la
from scipy import sparse as sps
from scipy.sparse import linalg as spla
# PAR 103

# Problem C

In [2]:
# Problem C
def inv_method(X, y, w):
    '''
    This method implements weighted least squares via the inversion method
    
    Inputs:
    X: An N x P array of features
    y: An N-array of responses
    w: An N-array of weights
    
    Outputs:
    Beta, A P-array solution vector for weighted least squares
    '''
    X_TW = X.T * w
    X_TWXinv = la.inv(np.dot(X_TW, X))
    X_TWy = np.dot(X_TW, y)
    Beta = np.dot(X_TWXinv, X_TWy)
    return Beta

In [3]:
# Problem C
def my_method(X, y, w):
    '''
    This method implements weighted least squares via the QR factorization method
    
    Inputs:
    X: An N x P array of features
    y: An N-array of responses
    w: An N-array of weights
    
    Outputs:
    Beta, A P-array solution vector for weighted least squares
    '''
    
    # Take W^.5X (note that this is a simple broadcast)
    w_5X = (w**.5).reshape(w.shape[0],1) * X
    # Compute the reduced QR factorization of W^.5 X
    Q, R = la.qr(w_5X, mode='economic')
    # Compute Q^T W^.5 y
    QTW_5y = np.dot(Q.T * w**.5,y)
    # Solve the upper triangular system
    beta = la.solve_triangular(R, QTW_5y)
    return beta

In [4]:
'''
Benchmarks for problem C
'''

N = 10000
P = 5000
X = np.random.rand(N, P) # Set a random array of samples
y = np.random.rand(N) # Set a random array of solutions
W = np.random.rand(N) # Set a random array of weights
#W = np.ones(N)


In [6]:
%%timeit 
'''
Check how long it takes for the QR factorzation method
Note that it should take somewhat longer than inversion (because the QR computation
cost is higher), but should be more stable
'''   
B_1 = my_method(X,y, W)

In [9]:

%%timeit

#Check how long it takes for the explicit inversion method
B_2 = inv_method(X,y,W)

1 loops, best of 3: 10.1 s per loop


In [31]:
# Note that floating point error comes into play when X is ill conditioned
N = 1000
P = 500
X = np.eye(N)
X[0,0] = 1e-12
X[50, 51] = 1e-12
y = np.random.rand(N) # Set a random array of solutions
W = np.random.rand(N) # Set a random array of weights
#W = np.ones(N)

B_1 = my_method(X,y, W) # QR attained beta
B_2 = inv_method(X,y,W) # Inversion attained beta
y_1 = X.dot(B_1) # Take the new approximate answer for qr
y_2 = X.dot(B_2) # Take the new approximate answer for inversion
print la.norm(y_1 - y) # Print the norm for QR
print la.norm(y_2 - y) # PRint the norm for inversion

8.39631705978e-16
1.40203118384e-15


Note that in the above code, there is a slight difference in the norms attained for the QR solution and the inverse solution (note that the QR solution is infintesimally closer). This demonstrates that even with SciPy's sophisticated inversion methods, floating point error is still introduced

# Problem D

In [32]:
# Problem D
def sps_wls(X, y, w):
    '''
    This implements the method of weighted least squares, but does so by exploiting the 
    sparsity of X.
    
    Inputs:
    X: An N x P sparse matrix (from scipy.sparse). 
    y: An N array of responses
    w: An N array of weights
    
    Outputs
    B: A dense P array that is the solution vector to the sparse system.
    '''
    
    # Begin by making sure that X is in csr format, which allows for matrix multiplication
    if X.format != 'csr':
        X = X.asformat('csr')
        
    # Set W to be a sparse diagonal matrix with values w
    W = sps.diags(w, format='csr')
    # Compute X_TW using the csr dot method.
    # Note that it might be better to use some kind of sparse broadcast, but I have no idea how
    X_TW = X.transpose().dot(W)
    # Again, use the sparse csr dot method to compute the linear algebra solver
    X_TWX = X_TW.dot(X)
    X_TWy = X_TW.dot(y)
    B = spla.spsolve(X_TWX, X_TWy)
    return B
    

In [33]:
# Problem D
# Generate test data for benchmarking on sparse X
N = 5000
P = 1000
X = sps.random(N, P)
dense_X = X.toarray()
w = np.random.rand(N)
y = np.random.rand(N)

In [34]:
%%timeit 
my_method(dense_X,y, w) # Benchmark QR

1 loops, best of 3: 605 ms per loop


In [35]:
%%timeit
inv_method(dense_X,y,w) # Benchmark Inverse method

10 loops, best of 3: 208 ms per loop


In [36]:
%%timeit
sps_wls(X,y,w) # Benchmark sparse method

10 loops, best of 3: 137 ms per loop


Note that from the above benchmarks we can see that our sparse implementation is faster than both our inverse and QR implementations when handling sparse matrices

In [38]:
# Check that the same results are attained for different densities
for d in np.arange(.1,1,.2):
    print "density =", d
    N = 5000
    P = 1000
    X = sps.random(N, P, density=d)
    dense_X = X.toarray()
    w = np.random.rand(N)
    y = np.random.rand(N)
    B1 = my_method(dense_X,y,w)
    B2 = inv_method(dense_X,y,w)
    B3 = sps_wls(X,y,w)
    # We check that the difference in the norms between the estimated and true
    # solutions are the same.
    print "QR method", la.norm(y - dense_X.dot(B1))
    print "Inverse method", la.norm(y - dense_X.dot(B2))
    print "Sparse method", la.norm(y - dense_X.dot(B3))
    print "\n"

density = 0.1
QR method 19.5278251967
Inverse method 19.5278251967
Sparse method 19.5278251967


density = 0.3
QR method 18.8543300148
Inverse method 18.8543300148
Sparse method 18.8543300148


density = 0.5
QR method 19.0359707725
Inverse method 19.0359707725
Sparse method 19.0359707725


density = 0.7
QR method 18.9738923548
Inverse method 18.9738923548
Sparse method 18.9738923548


density = 0.9
QR method 19.1990320605
Inverse method 19.1990320605
Sparse method 19.1990320605


